In [ ]:
!pip install -U google-genai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.7/236.7 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.41.0
    Uninstalling google-genai-1.41.0:
      Successfully uninstalled google-genai-1.41.0


In [ ]:
import os
os.environ["GEMINI_API_KEY"] = "your api key"


In [ ]:
# gemini_test.py
from google import genai
from google.genai import types

# If you already set GEMINI_API_KEY in your environment, client() will pick it up.
client = genai.Client()

# Simple text generation (same example from official quickstart)
response = client.models.generate_content(
    model="gemini-2.5-flash",                     # model name used in docs
    contents="Explain how AI works in a few words",
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0)  # set 0 to disable "thinking" (faster / cheaper)
    ),
)

print("=== Model reply ===")
print(response.text)


=== Model reply ===
Learning from data to make predictions.


In [ ]:
!pip install -U google-genai

import os
from google import genai

# 🔑 Set your Gemini API key
os.environ["GEMINI_API_KEY"] = "Your api key"

# Initialize client
client = genai.Client()

# 🧠 Take user input for the prompt
user_prompt = input("Enter your question or prompt for Gemini: ")

# Send request to Gemini model
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=user_prompt
)

# 🪄 Print model reply
print("\n=== Model reply ===")
print(response.text)


Enter your question or prompt for Gemini: explain all types of machine learning.

=== Model reply ===
Okay, let's break down the main types of machine learning and their key characteristics:

**1. Supervised Learning:**

*   **Core Idea:** This is the most common type of machine learning.  You provide the algorithm with labeled data, meaning you give it both the input features *and* the correct output (the "answer").  The algorithm learns a mapping function that can predict the output for new, unseen inputs.

*   **Analogy:** Think of teaching a child to recognize cats.  You show the child many pictures, each labeled "cat" or "not cat."  Eventually, the child learns to identify cats on their own.

*   **Key Characteristics:**
    *   **Labeled Data:**  Crucial.  This is what "supervises" the learning.
    *   **Goal:** To predict the output for new, unseen data.
    *   **Training Process:** The algorithm adjusts its internal parameters to minimize the difference between its prediction

In [ ]:
import os
from google import genai
import gradio as gr
import threading
import time
import sys

In [ ]:
# ---------- CONFIG ----------
# Replace with your key (or set it in another cell beforehand)
os.environ["GEMINI_API_KEY"] = "Your api key"
MODEL_NAME = "gemini-2.0-flash"   # change if you want a different model
# ----------------------------

# initialize client (reads GEMINI_API_KEY from env)
client = genai.Client()

# A flag to indicate app should stop (used for exit button)
stop_flag = {"exit": False}

# Function to call Gemini
def call_gemini(prompt: str, max_retries=1):
    """
    Call the Gemini model synchronously and return text.
    Simple wrapper; you can extend with configs (temperature, tokens, etc).
    """
    # Basic retry to handle transient errors
    last_err = None
    for attempt in range(max_retries + 1):
        try:
            resp = client.models.generate_content(
                model=MODEL_NAME,
                contents=prompt
            )
            # .text contains the textual reply in quickstart usage
            return resp.text
        except Exception as e:
            last_err = e
            # small backoff
            time.sleep(0.5 + attempt * 0.5)
    return f"[Error calling Gemini: {last_err}]"

# The main send function (called when Send button is pressed)
def send_prompt(user_prompt, chat_history):
    """
    user_prompt: the new prompt string
    chat_history: list of [user, assistant] pairs (Gradio Chatbot format)
    """
    if user_prompt is None or str(user_prompt).strip() == "":
        return chat_history, ""  # nothing to send

    # append user's message immediately
    chat_history = chat_history + [[user_prompt, "…"]]  # '…' as placeholder while waiting
    # update UI now and then perform the call
    # We perform blocking call here so user sees response after model returns
    reply = call_gemini(user_prompt)
    # replace the last assistant placeholder with actual reply
    chat_history[-1][1] = reply
    return chat_history, ""  # clear input box

# Clear button handler
def clear_conversation():
    return [], ""

# Exit button handler
def exit_app():
    # Set flag and then forcibly stop the notebook runtime process
    # NOTE: this will stop the Colab runtime for this notebook.
    stop_flag["exit"] = True
    # Gracefully give Gradio a moment, then exit the process
    def killer():
        time.sleep(0.5)
        try:
            # Try to stop programmatically
            print("Exiting Colab runtime (stop).")
            sys.exit(0)
        except SystemExit:
            # fallback to os._exit if sys.exit doesn't kill
            import os
            os._exit(0)
    threading.Thread(target=killer).start()
    return gr.update(visible=False), "Exiting..."

# Build Gradio Blocks UI
with gr.Blocks() as demo:
    gr.Markdown("## Gemini Chat (Colab)\nEnter your prompt below and press **Send**. Use **Clear** to reset chat. **Exit** will stop the notebook.")
    with gr.Row():
        with gr.Column(scale=3):
            chat = gr.Chatbot(value=[], label="Conversation")
            user_input = gr.Textbox(placeholder="Enter your prompt for Gemini here...", label="Prompt", lines=3)
            with gr.Row():
                send_btn = gr.Button("Send")
                clear_btn = gr.Button("Clear")
                exit_btn = gr.Button("Exit")
        with gr.Column(scale=1):
            # Optional side panel containing instructions or quick prompts
            gr.Markdown("### Quick help\n- Type a prompt and click **Send**.\n- After response you can type another prompt.\n- **Clear** resets conversation.\n- **Exit** stops the Colab runtime.")

    # wire buttons to functions
    send_btn.click(send_prompt, inputs=[user_input, chat], outputs=[chat, user_input])
    clear_btn.click(clear_conversation, outputs=[chat, user_input])
    exit_btn.click(exit_app, outputs=[demo, user_input])

# Launch the app inside Colab (no sharing link needed)
# In Colab, set share=False (default) and open inline.
demo.launch(share=False, inbrowser=False)

/tmp/ipython-input-1852029483.py:81: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat = gr.Chatbot(value=[], label="Conversation")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>